# Import Library

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

## Content

### 1. Import data
### 2. Data Consistency Check
### 3. Merge Data
### 4. Export Data

#### 1.  Import 'orders' and 'order_products_prior' data

In [2]:
# String for Main Project Folder
path = r'C:\Users\ifari\12-2022 Instacart Basket Analysis'

In [3]:
# String to import Data Folder containing orders_checked.csv
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), index_col = False)

In [4]:
# String to import Data Folder containing order_products_prior.csv
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products_prior.csv'), index_col = False)

In [5]:
# Drop extra index and resetting index
df_ords = df_ords.reset_index(drop = True)
df_ords = df_ords.drop(columns = ['Unnamed: 0'])

#### 2. Data Consistency Check on 'order_products_prior'

In [6]:
# Check for mixed datatypes in df_ords_prior
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col, ' mixed')
  else: print(col, ' consistent')

order_id  consistent
product_id  consistent
add_to_cart_order  consistent
reordered  consistent


In [7]:
# Check for missing values in the df_ords_prior
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [8]:
# Find duplicates in df_ords_prior
df_ords_prior_dups = df_ords_prior[df_ords_prior.duplicated()]
df_ords_prior_dups.size

0

> Data Consistency Checks were performed before merging. There are no mixed datatypes, missing values or duplicates. Thus, dataframe was exported into the prepared data and the merging process can begin.

#### 3. Merge 'orders' and 'order_product_prior'

In [9]:
# First 5 rows of df_ords
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [10]:
# Dimenstions of 'orders' dataframe
df_ords.shape

(3421083, 6)

In [11]:
# First 5 rows of df_ords_prior
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [12]:
# Dimenstions of 'order_products_prior' dataframe
df_ords_prior.shape

(32434489, 4)

In [13]:
# First 5 rows of the combined dataframes of 'orders' and 'order_products_prior'
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [14]:
# Dimenstions of merged dataframe
df_merged_large.shape

(32434489, 10)

In [15]:
# Frequency of the values in the '_merge' column
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

#### 4. Export Merged Dataframe

In [16]:
# Export data in pickle format
df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))